In [1]:
import os
import glob
import json
import pytz
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import dateutil.parser
from textblob import TextBlob
from datetime import datetime
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [106]:
news_df = pd.read_csv('./sent_with_preprocess_amzn_aapl.csv')
news_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment
0,0,0,0,2018-12-27 13:59:00-05:00,redliontrader.com,../Data/News/2018-12/news_0005405.json,compani tri time market buy back share go red ...,NaN,0.000000,0.019705
1,1,1,1,2018-12-06 06:02:00-05:00,businessinsider.com,../Data/News/2018-12/news_0001314.json,slowdown smartphon market due larg rise price ...,NaN,0.000000,0.007890
2,2,2,2,2018-12-27 12:05:00-05:00,sina.com.hk,../Data/News/2018-12/news_0004668.json,[],[],0.000000,0.000000
3,3,3,3,2018-12-20 16:05:00-05:00,marketwatch.com,../Data/News/2018-12/news_0004701.json,read data scientist crack code land new york t...,bad blood top amazon list 2 amazon best busi l...,0.020659,0.015055
4,4,4,4,2018-12-07 16:13:00-05:00,kcra.com,../Data/News/2018-12/news_0001237.json,comment didnt lift uncertainti wall street kee...,NaN,0.000000,0.025714
...,...,...,...,...,...,...,...,...,...,...
78050,78050,78050,78050,2018-10-17 20:00:00-04:00,money.ca,../Data/News/2018-10/news_0002911.json,fogchain nextgener fulli integr softwar develo...,NaN,0.000000,0.015444
78051,78051,78051,78051,2018-10-30 15:30:00-04:00,yahoo.com,../Data/News/2018-10/news_0004273.json,new macbook air arriv decad sinc former appl c...,NaN,0.000000,0.018492
78052,78052,78052,78052,2018-10-24 20:58:00-04:00,ivanhoff.com,../Data/News/2018-10/news_0004319.json,sinc correl extrem high correct market rather ...,NaN,0.000000,0.020449
78053,78053,78053,78053,2018-09-30 20:00:00-04:00,investors.com,../Data/News/2018-10/news_0000184.json,someth went wrong stock found support near 10w...,NaN,0.000000,-0.014336


In [0]:
import math
def removeNaN(text):
  if isinstance(text, float):
    if math.isnan(text):
      return []
  if isinstance(text, list):
    if text == []:
      return text
  if isinstance(text, str):
    return text

In [108]:
news_df['filteredtext_amzn'] = news_df['filteredtext_amzn'].apply(removeNaN)
news_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment
0,0,0,0,2018-12-27 13:59:00-05:00,redliontrader.com,../Data/News/2018-12/news_0005405.json,compani tri time market buy back share go red ...,[],0.000000,0.019705
1,1,1,1,2018-12-06 06:02:00-05:00,businessinsider.com,../Data/News/2018-12/news_0001314.json,slowdown smartphon market due larg rise price ...,[],0.000000,0.007890
2,2,2,2,2018-12-27 12:05:00-05:00,sina.com.hk,../Data/News/2018-12/news_0004668.json,[],[],0.000000,0.000000
3,3,3,3,2018-12-20 16:05:00-05:00,marketwatch.com,../Data/News/2018-12/news_0004701.json,read data scientist crack code land new york t...,bad blood top amazon list 2 amazon best busi l...,0.020659,0.015055
4,4,4,4,2018-12-07 16:13:00-05:00,kcra.com,../Data/News/2018-12/news_0001237.json,comment didnt lift uncertainti wall street kee...,[],0.000000,0.025714
...,...,...,...,...,...,...,...,...,...,...
78050,78050,78050,78050,2018-10-17 20:00:00-04:00,money.ca,../Data/News/2018-10/news_0002911.json,fogchain nextgener fulli integr softwar develo...,[],0.000000,0.015444
78051,78051,78051,78051,2018-10-30 15:30:00-04:00,yahoo.com,../Data/News/2018-10/news_0004273.json,new macbook air arriv decad sinc former appl c...,[],0.000000,0.018492
78052,78052,78052,78052,2018-10-24 20:58:00-04:00,ivanhoff.com,../Data/News/2018-10/news_0004319.json,sinc correl extrem high correct market rather ...,[],0.000000,0.020449
78053,78053,78053,78053,2018-09-30 20:00:00-04:00,investors.com,../Data/News/2018-10/news_0000184.json,someth went wrong stock found support near 10w...,[],0.000000,-0.014336


In [109]:
news_df[news_df['filteredtext_amzn'].astype(str) != '[]']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment
3,3,3,3,2018-12-20 16:05:00-05:00,marketwatch.com,../Data/News/2018-12/news_0004701.json,read data scientist crack code land new york t...,bad blood top amazon list 2 amazon best busi l...,0.020659,0.015055
9,9,9,9,2018-12-28 08:36:00-05:00,kake.com,../Data/News/2018-12/news_0005244.json,facebook fb 66 alphabet googl rose 56 netflix ...,facebook fb 66 alphabet googl rose 56 netflix ...,0.010670,-0.007564
20,20,20,20,2018-12-19 14:08:00-05:00,seekingalpha.com,../Data/News/2018-12/news_0004424.json,weve strong multiyear upcycl normal longer two...,weve strong multiyear upcycl normal longer two...,0.047438,0.047438
22,22,22,22,2018-12-25 19:00:00-05:00,investing.com,../Data/News/2018-12/news_0005459.json,sp index record new 52week high 194 new low na...,amazoncom inc amzno jump 612 percent report re...,0.019616,0.016389
24,24,24,24,2018-12-27 19:07:00-05:00,marketwatch.com,../Data/News/2018-12/news_0004799.json,see 4 biggest holiday stressor way copensom st...,see 4 biggest holiday stressor way copensom st...,0.015468,0.020087
...,...,...,...,...,...,...,...,...,...,...
78039,78039,78039,78039,2018-10-18 09:50:00-04:00,yahoo.com,../Data/News/2018-10/news_0003108.json,relat link peek market us stock futur fall ahe...,thesi stock market flash concern signal time a...,0.000340,0.002500
78040,78040,78040,78040,2018-10-29 20:00:00-04:00,morningstar.com,../Data/News/2018-10/news_0004130.json,stock enter decidedli volatil phase trade fuel...,share klatencor corp klac jump 68 earli trade ...,0.005029,0.008519
78045,78045,78045,78045,2018-10-05 11:29:00-04:00,yahoo.com,../Data/News/2018-10/news_0000012.json,broadli speak big tech dip bought facebook nas...,broadli speak big tech dip bought facebook nas...,-0.007910,-0.007910
78049,78049,78049,78049,2018-10-11 20:00:00-04:00,morningstar.com,../Data/News/2018-10/news_0002350.json,appl inc aapl rose 36 googlepar alphabet inc g...,share rose 27 appl inc aapl rose 36 googlepar ...,0.100000,0.032075


In [110]:
news_df['gmt_timestamp'] = pd.to_datetime(news_df['timestamp']).apply(lambda x: np.datetime64(x))
news_df = news_df.sort_values(by=['timestamp'])
news_df.index = (range(news_df.shape[0]))
#del news_df['Unnamed: 0']
news_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,gmt_timestamp
0,56483,56483,56483,2017-12-07 15:00:00-05:00,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.0,0.024667,2017-12-07 20:00:00
1,60250,60250,60250,2017-12-08 16:37:00-05:00,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.0,0.091103,2017-12-08 21:37:00
2,58563,58563,58563,2017-12-11 20:55:00-05:00,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.0,0.016494,2017-12-12 01:55:00
3,59776,59776,59776,2017-12-12 17:10:00-05:00,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.0,0.013467,2017-12-12 22:10:00
4,57779,57779,57779,2017-12-14 06:42:00-05:00,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.0,0.022093,2017-12-14 11:42:00
...,...,...,...,...,...,...,...,...,...,...,...
78050,67835,67835,67835,2019-02-07 15:52:00-05:00,eastmoney.com,../Data/News/2019-02/news_0001096.json,[],[],0.0,0.000000,2019-02-07 20:52:00
78051,67380,67380,67380,2019-02-07 16:06:00-05:00,stamfordadvocate.com,../Data/News/2019-02/news_0001336.json,turn surf web browser send littl bit metadata ...,[],0.0,0.015334,2019-02-07 21:06:00
78052,68409,68409,68409,2019-02-07 16:06:00-05:00,greenwichtime.com,../Data/News/2019-02/news_0001217.json,turn surf web browser send littl bit metadata ...,[],0.0,0.015334,2019-02-07 21:06:00
78053,68294,68294,68294,2019-02-07 17:05:00-05:00,nasdaq.com,../Data/News/2019-02/news_0001333.json,new york nov 30 2018 globe newswir new indepen...,[],0.0,0.014382,2019-02-07 22:05:00


**Change ./APPLE1440.csv to respective time interval csv**

In [111]:
time_zone = pytz.timezone('US/Eastern')
headers = ['date', 'time', 'open', 'close', 'low', 'high', 'volume']
df = pd.read_csv("./APPLE1440.csv", header=None, names = headers)
df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time']).dt.tz_localize(time_zone)
df['gmt_timestamp'] = df['timestamp'].apply(lambda x: np.datetime64(x))
df['label'] = df['open'] - df['open'].shift(1)
df = df.dropna(axis=0)
df['label'] = df['label'].apply(lambda x: -1 if x<0 else 1)
del df['date'], df['time']
print(news_df.shape)
news_df = news_df[news_df.gmt_timestamp < df.iloc[-1]['gmt_timestamp']]
print(news_df.shape)

(78055, 11)
(76826, 11)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """


In [112]:
df.head()

,open,close,low,high,volume,timestamp,gmt_timestamp,label
1,93.973,94.546,93.611,93.993,4787,2014-07-01 00:00:00-04:00,2014-07-01 04:00:00,1
2,94.344,94.515,93.540,93.902,4476,2014-07-02 00:00:00-04:00,2014-07-02 04:00:00,1
3,94.113,94.545,93.661,94.495,3454,2014-07-03 00:00:00-04:00,2014-07-03 04:00:00,-1
4,94.936,96.423,94.936,96.403,4638,2014-07-07 00:00:00-04:00,2014-07-07 04:00:00,1
5,96.790,97.242,94.358,95.795,7421,2014-07-08 00:00:00-04:00,2014-07-08 04:00:00,1


In [113]:
df['timestamp']

1      2014-07-01 00:00:00-04:00
2      2014-07-02 00:00:00-04:00
3      2014-07-03 00:00:00-04:00
4      2014-07-07 00:00:00-04:00
5      2014-07-08 00:00:00-04:00
                  ...           
1138   2019-01-28 00:00:00-05:00
1139   2019-01-29 00:00:00-05:00
1140   2019-01-30 00:00:00-05:00
1141   2019-01-31 00:00:00-05:00
1142   2019-02-01 00:00:00-05:00
Name: timestamp, Length: 1142, dtype: datetime64[ns, US/Eastern]

In [0]:
start = news_df['gmt_timestamp'][0]

In [115]:
news_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'timestamp', 'site',
       'news_file_loc', 'filteredtext_aapl', 'filteredtext_amzn',
       'amzn_sentiment', 'aapl_sentiment', 'gmt_timestamp'],
      dtype='object')

In [0]:
df = df[df['gmt_timestamp'] >= start]

In [117]:
df.index = range(df.shape[0])
df['gmt_timestamp']

0     2017-12-08 05:00:00
1     2017-12-11 05:00:00
2     2017-12-12 05:00:00
3     2017-12-13 05:00:00
4     2017-12-14 05:00:00
              ...        
283   2019-01-28 05:00:00
284   2019-01-29 05:00:00
285   2019-01-30 05:00:00
286   2019-01-31 05:00:00
287   2019-02-01 05:00:00
Name: gmt_timestamp, Length: 288, dtype: datetime64[ns]

In [0]:
end  = np.datetime64(df['gmt_timestamp'][0])
news_times = news_df['gmt_timestamp'].values
#end = pd.to_datetime(end)

In [119]:
timestamplen = len(df['gmt_timestamp'].values)
i=0; k=-1
stock_times = []
for i,time in enumerate(news_times):
  #time = np.datetime64(time)
  if time <= end:
    stock_times.append(end)
    print (time, end)
  else:
    if k+1<timestamplen:
      k = k+1
      temp = df['gmt_timestamp'].values[k]
      while(time > temp):
        if k+1<timestamplen:
          k = k+1
          #start = df['timestamp'].values[k]
          temp  = df['gmt_timestamp'].values[k]
        else:
          break
    end = temp
    stock_times.append(end)
    print (time, end)

Streaming output truncated to the last 5000 lines.
2019-01-09T10:00:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:01:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:15:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:30:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:30:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:30:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:30:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:30:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:33:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:35:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:45:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:45:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:50:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T10:51:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T11:00:00.000000000 2019-01-10T05:00:00.000000000
2019-01-09T11:08:00.000000000 2019-01-10T05:00:00

In [120]:
len(stock_times)

76826

In [121]:
end

numpy.datetime64('2019-02-01T05:00:00.000000000')

In [122]:
news_times[i]

numpy.datetime64('2019-02-01T04:59:00.000000000')

In [123]:
stock_times[-1]

numpy.datetime64('2019-02-01T05:00:00.000000000')

In [124]:
news_df['gmt_timestamp']

0       2017-12-07 20:00:00
1       2017-12-08 21:37:00
2       2017-12-12 01:55:00
3       2017-12-12 22:10:00
4       2017-12-14 11:42:00
                ...        
76821   2019-02-01 04:47:00
76822   2019-02-01 04:57:00
76823   2019-02-01 04:57:00
76824   2019-02-01 04:57:00
76825   2019-02-01 04:59:00
Name: gmt_timestamp, Length: 76826, dtype: datetime64[ns]

In [125]:
df.shape

(288, 8)

In [126]:
df.iloc[-1]['gmt_timestamp']

Timestamp('2019-02-01 05:00:00')

In [0]:
news_df = news_df[news_df.gmt_timestamp < df.iloc[-1]['gmt_timestamp']]

In [128]:
news_df.shape

(76826, 11)

In [0]:
news_df['stock_time'] = stock_times

In [130]:
news_df[news_df.gmt_timestamp > news_df.stock_time]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,gmt_timestamp,stock_time


In [131]:
news_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,gmt_timestamp,stock_time
0,56483,56483,56483,2017-12-07 15:00:00-05:00,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.000000,0.024667,2017-12-07 20:00:00,2017-12-08 05:00:00
1,60250,60250,60250,2017-12-08 16:37:00-05:00,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.000000,0.091103,2017-12-08 21:37:00,2017-12-11 05:00:00
2,58563,58563,58563,2017-12-11 20:55:00-05:00,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.000000,0.016494,2017-12-12 01:55:00,2017-12-12 05:00:00
3,59776,59776,59776,2017-12-12 17:10:00-05:00,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.000000,0.013467,2017-12-12 22:10:00,2017-12-13 05:00:00
4,57779,57779,57779,2017-12-14 06:42:00-05:00,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.000000,0.022093,2017-12-14 11:42:00,2017-12-15 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
76821,68498,68498,68498,2019-01-31 23:47:00-05:00,seekingalpha.com,../Data/News/2019-02/news_0001145.json,monitor 36 trillion proxi market known nyse fa...,monitor 36 trillion proxi market known nyse fa...,0.028095,0.028095,2019-02-01 04:47:00,2019-02-01 05:00:00
76822,68572,68572,68572,2019-01-31 23:57:00-05:00,stamfordadvocate.com,../Data/News/2019-02/news_0000438.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00
76823,68026,68026,68026,2019-01-31 23:57:00-05:00,sfgate.com,../Data/News/2019-02/news_0000126.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00
76824,67397,67397,67397,2019-01-31 23:57:00-05:00,greenwichtime.com,../Data/News/2019-02/news_0000434.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00


In [0]:
final_df = news_df.merge(df, left_on='stock_time', right_on='gmt_timestamp')

In [133]:
final_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,timestamp_x,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,gmt_timestamp_x,stock_time,open,close,low,high,volume,timestamp_y,gmt_timestamp_y,label
0,56483,56483,56483,2017-12-07 15:00:00-05:00,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.0,0.024667,2017-12-07 20:00:00,2017-12-08 05:00:00,169.86,170.98,168.79,169.31,49427,2017-12-08 00:00:00-05:00,2017-12-08 05:00:00,1
1,60250,60250,60250,2017-12-08 16:37:00-05:00,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.0,0.091103,2017-12-08 21:37:00,2017-12-11 05:00:00,169.39,172.88,168.77,172.67,53369,2017-12-11 00:00:00-05:00,2017-12-11 05:00:00,-1
2,58563,58563,58563,2017-12-11 20:55:00-05:00,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.0,0.016494,2017-12-12 01:55:00,2017-12-12 05:00:00,172.40,172.49,171.43,171.65,43289,2017-12-12 00:00:00-05:00,2017-12-12 05:00:00,1
3,59776,59776,59776,2017-12-12 17:10:00-05:00,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.0,0.013467,2017-12-12 22:10:00,2017-12-13 05:00:00,171.62,173.52,171.62,172.31,49072,2017-12-13 00:00:00-05:00,2017-12-13 05:00:00,-1
4,57779,57779,57779,2017-12-14 06:42:00-05:00,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.0,0.022093,2017-12-14 11:42:00,2017-12-15 05:00:00,172.55,174.13,172.38,173.85,52954,2017-12-15 00:00:00-05:00,2017-12-15 05:00:00,1


In [0]:
del final_df['timestamp_x'], final_df['timestamp_y'], final_df['gmt_timestamp_y']

In [0]:
column_list = list(final_df.columns)

In [0]:
for i,name in enumerate(column_list):
    if name == 'gmt_timestamp_x':
        column_list[i] = 'news_timestamp'

In [0]:
final_df.columns = column_list

In [138]:
final_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,news_timestamp,stock_time,open,close,low,high,volume,label
0,56483,56483,56483,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.000000,0.024667,2017-12-07 20:00:00,2017-12-08 05:00:00,169.86,170.98,168.79,169.31,49427,1
1,60250,60250,60250,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.000000,0.091103,2017-12-08 21:37:00,2017-12-11 05:00:00,169.39,172.88,168.77,172.67,53369,-1
2,58563,58563,58563,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.000000,0.016494,2017-12-12 01:55:00,2017-12-12 05:00:00,172.40,172.49,171.43,171.65,43289,1
3,59776,59776,59776,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.000000,0.013467,2017-12-12 22:10:00,2017-12-13 05:00:00,171.62,173.52,171.62,172.31,49072,-1
4,57779,57779,57779,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.000000,0.022093,2017-12-14 11:42:00,2017-12-15 05:00:00,172.55,174.13,172.38,173.85,52954,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76821,68498,68498,68498,seekingalpha.com,../Data/News/2019-02/news_0001145.json,monitor 36 trillion proxi market known nyse fa...,monitor 36 trillion proxi market known nyse fa...,0.028095,0.028095,2019-02-01 04:47:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76822,68572,68572,68572,stamfordadvocate.com,../Data/News/2019-02/news_0000438.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76823,68026,68026,68026,sfgate.com,../Data/News/2019-02/news_0000126.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76824,67397,67397,67397,greenwichtime.com,../Data/News/2019-02/news_0000434.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1


In [139]:
final_df[final_df['label'] == 1].shape

(39390, 17)

In [140]:
final_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,news_timestamp,stock_time,open,close,low,high,volume,label
0,56483,56483,56483,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.0,0.024667,2017-12-07 20:00:00,2017-12-08 05:00:00,169.86,170.98,168.79,169.31,49427,1
1,60250,60250,60250,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.0,0.091103,2017-12-08 21:37:00,2017-12-11 05:00:00,169.39,172.88,168.77,172.67,53369,-1
2,58563,58563,58563,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.0,0.016494,2017-12-12 01:55:00,2017-12-12 05:00:00,172.40,172.49,171.43,171.65,43289,1
3,59776,59776,59776,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.0,0.013467,2017-12-12 22:10:00,2017-12-13 05:00:00,171.62,173.52,171.62,172.31,49072,-1
4,57779,57779,57779,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.0,0.022093,2017-12-14 11:42:00,2017-12-15 05:00:00,172.55,174.13,172.38,173.85,52954,1


In [172]:
final_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,site,news_file_loc,filteredtext_aapl,filteredtext_amzn,amzn_sentiment,aapl_sentiment,news_timestamp,stock_time,open,close,low,high,volume,label
0,56483,56483,56483,businesswire.com,../Data/News/2018-01/news_0000265.json,2018 nape global busi confer held februari 7 2...,[],0.000000,0.024667,2017-12-07 20:00:00,2017-12-08 05:00:00,169.86,170.98,168.79,169.31,49427,1
1,60250,60250,60250,yahoo.com,../Data/News/2018-01/news_0001827.json,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,[],0.000000,0.091103,2017-12-08 21:37:00,2017-12-11 05:00:00,169.39,172.88,168.77,172.67,53369,-1
2,58563,58563,58563,yahoo.com,../Data/News/2018-01/news_0000282.json,web publish would wise rememb arent kind bad a...,[],0.000000,0.016494,2017-12-12 01:55:00,2017-12-12 05:00:00,172.40,172.49,171.43,171.65,43289,1
3,59776,59776,59776,investingnews.com,../Data/News/2018-01/news_0001849.json,fact cobalt mine mainli byproduct metal copper...,[],0.000000,0.013467,2017-12-12 22:10:00,2017-12-13 05:00:00,171.62,173.52,171.62,172.31,49072,-1
4,57779,57779,57779,seekingalpha.com,../Data/News/2018-01/news_0001855.json,snap nyse snap cramer fan cramer trust doesnt ...,[],0.000000,0.022093,2017-12-14 11:42:00,2017-12-15 05:00:00,172.55,174.13,172.38,173.85,52954,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76821,68498,68498,68498,seekingalpha.com,../Data/News/2019-02/news_0001145.json,monitor 36 trillion proxi market known nyse fa...,monitor 36 trillion proxi market known nyse fa...,0.028095,0.028095,2019-02-01 04:47:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76822,68572,68572,68572,stamfordadvocate.com,../Data/News/2019-02/news_0000438.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76823,68026,68026,68026,sfgate.com,../Data/News/2019-02/news_0000126.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1
76824,67397,67397,67397,greenwichtime.com,../Data/News/2019-02/news_0000434.json,certif allow facebook distribut intern app emp...,[],0.000000,0.008725,2019-02-01 04:57:00,2019-02-01 05:00:00,166.10,168.96,165.90,166.13,51990,1


In [0]:
final_df.to_csv('./final_dataset_joined_1440.csv')

In [0]:
res = final_df.groupby('stock_time')

In [0]:
keys = final_df['stock_time'].unique()

# **Change to filteredtext_amzn when using amazon stocks**

# **Change to amzn_sentiment when using amazon stocks**

In [0]:
dtext = dict()
for key in keys:
    # change t0 filteredtext_amzn when using amazon stocks
    dtext[key] = res.get_group(key)['filteredtext_aapl'].to_list()
dsent = dict()
for key in keys:
    # change t0 amzn_sentiment when using amazon stocks
    dsent[key] = res.get_group(key)['aapl_sentiment'].to_list()
dlabel = dict()
for key in keys:
    dlabel[key] = res.get_group(key)['label'].to_list()

In [0]:
for key in keys:
    dlabel[key] = sum(dlabel[key])/len(dlabel[key])
for key in keys:
    dsent[key] = sum(dsent[key])/len(dsent[key])

In [0]:
for key in keys:
  finaltextset = []
  for text in dtext[key]:
    if text != '[]':
      finaltextset.append(text)
  finaltext = ''
  for text in finaltextset:
    finaltext += str(text) + ' '
  dtext[key] = finaltext

In [0]:
finaldict = dict()
for key in keys:
    finaldict[key] = [dtext[key], dsent[key], dlabel[key]]

In [0]:
text = []
sent = []
label = []
dictkey = []
testdict = dict()
for key in keys:
    dictkey.append(key)
    text.append(finaldict[key][0])
    sent.append(finaldict[key][1])
    label.append(finaldict[key][2])

In [0]:
testdict['text'] = text
testdict['sentiment'] = sent
testdict['label'] = label
testdict['stock_time'] = dictkey

# **Change name to df_{time_interval}**

In [0]:
# change name to df_{time_interval}
df_1440 = pd.DataFrame(testdict)

In [197]:
df_1440

,text,sentiment,label,stock_time
0,2018 nape global busi confer held februari 7 2...,0.024667,1.0,2017-12-08 05:00:00
1,725 s8 25 iphon 8 perhap ngalaxi s8 best smart...,0.091103,-1.0,2017-12-11 05:00:00
2,web publish would wise rememb arent kind bad a...,0.016494,1.0,2017-12-12 05:00:00
3,fact cobalt mine mainli byproduct metal copper...,0.013467,-1.0,2017-12-13 05:00:00
4,snap nyse snap cramer fan cramer trust doesnt ...,0.031292,1.0,2017-12-15 05:00:00
...,...,...,...,...
281,2019 tast year got scorcher blaze hot melbourn...,0.012952,1.0,2019-01-28 05:00:00
282,aapl topic sever report compani offer iphon li...,0.011818,1.0,2019-01-29 05:00:00
283,httpswwwyoutubecomwatchv6enwkf1sqog tag aapl a...,0.009776,1.0,2019-01-30 05:00:00
284,compani offer iphon line smartphon ipad line m...,0.009638,1.0,2019-01-31 05:00:00


# **Change name to finaldf{time_interval}_stock**

In [0]:
df_1440.to_csv('./final1440_aapl.csv')